In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm

import re 

def is_russian(text):
    ans = True
    for i in text:
        ans = ans & bool(re.search('[а-яА-Я]', i))
    return bool(ans)

In [3]:
import csv

words = {}

with open('words.csv', newline='') as file:
    buffer = csv.DictReader(file)
    for r in tqdm(buffer):
        if is_russian(r['Id']):
            words[r['Id']] = int(r['Freq'])
            
len(words)

724512it [00:11, 63908.20it/s]


501381

In [4]:
from itertools import groupby
from Levenshtein import editops
expected_train_values = set()
train_values = list()
map1 = {}
map2 = {}
map3 = {}
replace_key = None
with open('train.csv', newline='') as file:
    buffer = csv.DictReader(file)
    for r in tqdm(buffer):
        if is_russian(r['Expected']):
            expected_train_values.add(r['Expected'])
            train_values.append((r['Id'], r['Expected']))
            edits = editops(r['Id'], r['Expected'])
            last_edit = None
            letters = dict()
            for i in r['Expected']:
                letters[i] = 0
            for i in r['Expected']:
                letters[i] += 1

            for edit in edits:
                edit = list(edit)
#                print(edit[0], edit[1])
                if edit[0] == 'replace':
                    replace_key = edit[0]
                    edit[1] = r['Id'][edit[1]]
                    edit[2] = r['Expected'][edit[2]]
                    if letters.get(edit[2]) is not None:
                        letters[edit[2]] -= 1
                        if letters[edit[2]] == 0:
                            del letters[edit[2]]
                elif edit[0] == 'delete':
                    edit[1] = r['Id'][edit[1]]
                    edit[2] = None
                elif edit[0] == 'insert':
                    edit[1] = r['Expected'][edit[2]]
                    if letters.get(edit[1]) is not None:
                        letters[edit[1]] -= 1
                        if letters[edit[1]] == 0:
                            del letters[edit[1]]
                    edit[2] = None

                map1.setdefault(edit[0], 0)
                map1[edit[0]] += 1
                map2.setdefault(str(edit), 0)
                map2[str(edit)] += 1
                map3.setdefault((str(edit), last_edit), 0)
                map3[(str(edit), last_edit)] += 1
                last_edit = str(edit)
            for i in letters:
                cnt = letters[i]
                map1.setdefault('replace', 0)
                map1['replace'] += cnt
                for j in range(cnt):
                    l = str(['replace', i, i])
                    map2.setdefault(l, 0)
                    map2[l] += 1

            
train_values = [el for el, _ in groupby(train_values)]          
len(train_values)

362256it [00:12, 29217.58it/s]


251385

In [105]:
len(map2)

826

In [10]:
train = []
with open('no_fix.submission.csv') as file:
    buffer = csv.DictReader(file)
    for r in tqdm(buffer):
        train.append(r['Id'])

362256it [00:01, 274613.60it/s]


In [61]:
import operator
from collections import OrderedDict
from itertools import groupby

class TrieNode:
    def __init__(self):
        self.end = False
        self.children = {}
        self.prob = 0
        self.word = None

    def all_words(self, prefix):
        if self.end:
            yield prefix

        for letter, child in self.children.items():
            yield from child.all_words(prefix + letter)

class PrefixTree:
    def __init__(self):
        self.root = TrieNode()
        self.nodes = []

    def insert(self, word):
        curr = self.root
        for letter in word:
            node = curr.children.get(letter)
            if not node:
                node = TrieNode()
                curr.children[letter] = node
            curr = node
        curr.end = True
        curr.prob = words[word] if words.get(word) is not None else 0
        curr.word = word

    def search(self, word):
        curr = self.root
        for letter in word:
            node = curr.children.get(letter)
            if not node:
                return False
            curr = node
        return curr.end
    
    def all_words_beginning_with_prefix(self, prefix):
        cur = self.root
        for c in prefix:
            cur = cur.children.get(c)
            if cur is None:
                return  # No words with given prefix

        yield from cur.all_words(prefix)
        
    def dfs(self, word, node, prev_corr=None, pos=0, cor_num=0):
        if pos == len(word):
            if node.end:
                self.nodes.append((node.prob, node.word))
            return
        if cor_num > 1:
            if node.end and (pos == len(word)):
                self.nodes.append((node.prob, node.word))
            return
        if len(node.children) == 0 and (pos == len(word)):
            self.nodes.append((node.prob, node.word))
            return
        if node.children.get(word[pos]) is None:
            if node.end and (pos == len(word)):
                self.nodes.append((node.prob, node.word))
            return
        corrections = []
        if pos >= len(word)-3:
            corrections.append(('replace', word[pos], word[pos]))
        else:
            for ch in node.children:
                corrections.append(('replace', word[pos], ch))
            
        p_cor = dict()
        for c in corrections:
            c = list(c)
            p_cor.setdefault(str(c), 0)
            p_cor[str(c)] = map2[str(c)] if map2.get(str(c)) is not None else 0
        sorted_p_cor = OrderedDict(sorted(p_cor.items(), key=lambda kv: kv[1], reverse=True))
        i = 0
        for corr in sorted_p_cor:
            corr = eval(corr)
            if i > 3:
                break
            i += 1
            if corr[0] == 'replace':
                new_node = node.children[corr[2]]
                if corr[1] == corr[2]:
                    self.dfs(word, new_node, str(corr), pos + 1, cor_num)
                else:
                    self.dfs(word, new_node, str(corr), pos + 1, cor_num + 1)



In [62]:
trie = PrefixTree()
for word in (words.keys()):
    trie.insert(word)
    

In [72]:
from Levenshtein import distance
ans = []
with open('no_fix.submission.csv') as file:
    buffer = csv.DictReader(file)
    i = 0
    sum_dist = 0
    for r in (buffer):
        if not is_russian(r['Id']):
            ans.append((r['Id'], r['Id']))
            sum_dist += distance(r['Predicted'], r['Id'])
            i += 1
            continue
        trie.nodes = []
        trie.dfs(r['Id'], trie.root)
        mylist = list(dict.fromkeys(trie.nodes))
        best_list = sorted(mylist, key = lambda x: x[0], reverse=True)
        best_match = best_list[0][1] if len(best_list) > 1 else None
        best_match2 = best_list[1][1] if len(best_list) > 1 else None
        if best_match is None:
            best_match = r['Id']
        elif words[best_match] / words[r['Id']] < 5.0:
            best_match = r['Id']
        elif best_match2 is not None:
            if words[best_match] / words[best_match2] < 3.0:
                best_match = r['Id']
        sum_dist += distance(r['Predicted'], best_match)
        ans.append((r['Id'], best_match))
        i += 1
        if i % 1000 == 0:
            print(i)
    print(sum_dist / i)

2000
3000
5000
7000
8000
9000
10000
12000
13000
15000
17000
19000
21000
23000
25000
26000
27000
28000
29000
31000
34000
38000
39000
40000
41000
42000
43000
44000
48000
49000
52000
53000
54000
56000
57000
59000
60000
61000
62000
63000
66000
67000
68000
70000
71000
73000
74000
76000
77000
82000
83000
84000
85000
87000
89000
91000
92000
93000
95000
99000
101000
103000
105000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
119000
120000
121000
122000
124000
125000
128000
129000
131000
132000
133000
134000
135000
136000
137000
138000
140000
141000
143000
146000
147000
148000
149000
150000
151000
152000
153000
156000
157000
160000
161000
162000
163000
168000
169000
172000
175000
176000
177000
178000
181000
184000
185000
186000
188000
189000
190000
191000
192000
193000
194000
195000
196000
197000
198000
199000
200000
201000
202000
205000
206000
208000
210000
212000
214000
215000
218000
220000
221000
222000
223000
224000
225000
226000
227000
228000
230000
231000
232000
23

In [73]:
with open('my_submission2.csv', 'w', newline='') as file:
    buffer = csv.writer(file)
    buffer.writerow(['Id', 'Predicted'])
    for orig, pred in ans:
        buffer.writerow([orig, pred])